In [3]:
#Trains neural network using GSE63990_series_matrix.txt and homebrew code

In [6]:
#imports all needed modules 
import NotebookFinder
import data_cleaner as dclean
import numpy as np
import pandas as pd
import MLP_Clean as NN
import gene_selection as gs

#import feature selection modules
import sklearn.feature_selection 

In [2]:
#data organization/reformatting of GSE63990_series_matrix.txt 

#returns characteristics = X
        #expression = Y
        # 'non-infectious illness': 0, 'bacterial': 1, 'viral': 2

#call on data_cleaner to open and formate GSE63990_series_matrix
data = dclean.clean_GSE_series_matrix('GSE63990_series_matrix.txt')
# X_train, Y_train, X_test, Y_test = gs.select(data, 10)



reading file...
Dropping Series_title
Dropping Series_geo_accession
Dropping Series_status
Dropping Series_submission_date
Dropping Series_last_update_date
Dropping Series_pubmed_id
Dropping Series_summary
Dropping Series_overall_design
Dropping Series_type
Dropping Series_contributor
Dropping Series_sample_id
Dropping Series_contact_name
Dropping Series_contact_email
Dropping Series_contact_department
Dropping Series_contact_institute
Dropping Series_contact_address
Dropping Series_contact_city
Dropping Series_contact_state
Dropping Series_contact_zip/postal_code
Dropping Series_contact_country
Dropping Series_supplementary_file
Dropping Series_platform_id
Dropping Series_platform_taxid
Dropping Series_sample_taxid
Dropping Series_relation
Converting Sample_characteristics_ch1


In [3]:
def round_param(parameters):
    #rounding to nearest integer doesn't work
    #round to 1 decimal place does work
    W1=np.round(parameters['W1'],1)
    W2=np.round(parameters['W2'],1)
    b1=np.round(parameters['b1'],1)
    b2=np.round(parameters['b2'],1)   
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return(parameters)

In [4]:
#find the best models without rounding
hidden_layer_sizes = np.arange(1,10)
gene_list = np.arange(1,20)
for gene_number in gene_list:
    X_train, Y_train, X_test, Y_test = gs.select(data, gene_number)
    for seed in seed_list:
        for n_h in hidden_layer_sizes:
            parameters = NN.nn_model(X_train, Y_train, n_h, num_iterations = 500, seed=seed)
            #round
            parameters = round_param(parameters)
            predictions = NN.predict(parameters, X_test)
            labels = np.argmax(Y_test, axis = 0)
            t = (labels-predictions)
            accuracy = np.sum(np.where(t == 0, 1, 0))/Y_test.shape[1]*100
            if accuracy > 75:
                print ("with {} gene(s)".format(gene_number))
                print ("random seed:", seed)
                print ("Accuracy for {} hidden units: {} %".format(n_h, accuracy))
                parameters

/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
with 1 gene(s)
random seed: 8
Accuracy for 5 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 20
Accuracy for 5 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 20
Accuracy for 8 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 27
Accuracy for 5 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 28
Accuracy for 7 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 42
Accuracy for 4 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 42
Accuracy for 6 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 46
Accuracy for 6 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 51
Accuracy for 6 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 57
Accuracy for 5 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 59
Accuracy for 8 hidden units: 78.57142857142857 %
with 1 gene(s)
random seed: 62
Accu

/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
6852                 207329_at"  51.908816
with 2 gene(s)
random seed: 17
Accuracy for 7 hidden units: 78.57142857142857 %
with 2 gene(s)
random seed: 45
Accuracy for 5 hidden units: 78.57142857142857 %
with 2 gene(s)
random seed: 65
Accuracy for 8 hidden units: 78.57142857142857 %
with 2 gene(s)
random seed: 82
Accuracy for 7 hidden units: 78.57142857142857 %
with 2 gene(s)
random seed: 87
Accuracy for 8 hidden units: 82.14285714285714 %


/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


...

with 18 gene(s)
random seed: 59
Accuracy for 4 hidden units: 85.71428571428571 %
with 18 gene(s)
random seed: 75
Accuracy for 5 hidden units: 78.57142857142857 %


/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
6852                 207329_at"  51.908816
21443              222084_s_at"  45.796933
12152              212768_s_at"  45.511495
10952              211548_s_at"  39.885743
3439               203913_s_at"  39.213689
19338              219975_x_at"  38.786768
9477                 209992_at"  36.792002
5370                 205844_at"  36.757702
5702               206177_s_at"  35.538623
19032                219669_at"  35.401498
3440               203914_x_at"  35.133532
8786               209293_x_at"  34.937390
22240  AFFX-HUMRGE/M10098_M_at"  34.897467
16040              216671_x_at"  31.861675
9445                 209960_at"  28.876673
16806                217442_at"  28.460255
2680                 203153_at"  27.864647
5975                 206450_at"  27.230384


MLP_Clean.ipynb:18: RuntimeWarning: divide by zero encountered in log
  "outputs": [],
MLP_Clean.ipynb:18: RuntimeWarning: invalid value encountered in multiply
  "outputs": [],


with 19 gene(s)
random seed: 54
Accuracy for 8 hidden units: 78.57142857142857 %
with 19 gene(s)
random seed: 66
Accuracy for 5 hidden units: 78.57142857142857 %
with 19 gene(s)
random seed: 94
Accuracy for 7 hidden units: 82.14285714285714 %


In [26]:
# try select models with rounding
randomseed = 61
number_hidden = 5
gene_number = 9

X_train, Y_train, X_test, Y_test = gs.select(data, gene_number)
parameters = NN.nn_model(X_train, Y_train, n_h = number_hidden, num_iterations = 500, print_cost=True, seed=randomseed)
parameters = round_param(parameters)
predictions = NN.predict(parameters, X_train)
labels = np.argmax(Y_train, axis = 0)
t = (labels-predictions)      
accuracy = np.sum(np.where(t == 0, 1, 0))/Y_train.shape[1]*100
print('train accuracy:', accuracy,'%')
predictions = NN.predict(parameters, X_test)
labels = np.argmax(Y_test, axis = 0)
t = (labels-predictions)      
accuracy = np.sum(np.where(t == 0, 1, 0))/Y_test.shape[1]*100
print('test accuracy:', accuracy,'%')

/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
6852                 207329_at"  51.908816
21443              222084_s_at"  45.796933
12152              212768_s_at"  45.511495
10952              211548_s_at"  39.885743
3439               203913_s_at"  39.213689
19338              219975_x_at"  38.786768
9477                 209992_at"  36.792002
5370                 205844_at"  36.757702
Cost after iteration 0: 1048.368085
train accuracy: 65.87301587301587 %
test accuracy: 82.14285714285714 %


In [27]:
Y_test = 

array([[0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
        1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1.]])

In [24]:
#try with other random splits of the dataset
X_train, Y_train, X_test, Y_test = gs.select(data, gene_number, seed=50)
predictions = NN.predict(parameters, X_train)
labels = np.argmax(Y_train, axis = 0)
t = (labels-predictions)      
accuracy = np.sum(np.where(t == 0, 1, 0))/Y_train.shape[1]*100
print('train accuracy:', accuracy,'%')
predictions = NN.predict(parameters, X_test)
labels = np.argmax(Y_test, axis = 0)
t = (labels-predictions)      
accuracy = np.sum(np.where(t == 0, 1, 0))/Y_test.shape[1]*100
print('test accuracy:', accuracy,'%')

/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


                           Gene      Score
22239  AFFX-HUMRGE/M10098_5_at"  62.413376
6852                 207329_at"  51.908816
train accuracy: 67.46031746031747 %
test accuracy: 46.42857142857143 %


In [16]:
#try different feature selection methods
X_train, Y_train, X_test, Y_test = gs.select(data, 20, score_func = sklearn.feature_selection.f_classif)

/home/ray/.local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


ValueError: bad input shape (280, 3)

In [ ]:
# Gene      Score
# 22239  AFFX-HUMRGE/M10098_5_at"  62.413376
# 6852                 207329_at"  51.908816
# 21443              222084_s_at"  45.796933
# 12152              212768_s_at"  45.511495
# 10952              211548_s_at"  39.885743
# 3439               203913_s_at"  39.213689
# 19338              219975_x_at"  38.786768
# 9477                 209992_at"  36.792002
# 5370                 205844_at"  36.757702
# 5702               206177_s_at"  35.538623
# 19032                219669_at"  35.401498
# 3440               203914_x_at"  35.133532
# 8786               209293_x_at"  34.937390
# 22240  AFFX-HUMRGE/M10098_M_at"  34.897467
# 16040              216671_x_at"  31.861675
# 9445                 209960_at"  28.876673
# 16806                217442_at"  28.460255
# 2680                 203153_at"  27.864647
# 5975                 206450_at"  27.230384